# Data Frame 3A


In [4]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
%pip install wbdata
import wbdata
%pip install cufflinks
import cufflinks as cf
# %pip install geopandas
import geopandas as gpd
import rasterio
import rasterio.plot
import IPython
cf.go_offline()

  Using cached wbdata-1.0.0-py3-none-any.whl.metadata (2.6 kB)
  Using cached appdirs-1.4.4-py2.py3-none-any.whl.metadata (9.0 kB)
  Using cached backoff-2.2.1-py3-none-any.whl.metadata (14 kB)
  Using cached dateparser-1.2.1-py3-none-any.whl.metadata (29 kB)
  Using cached shelved_cache-0.3.1-py3-none-any.whl.metadata (4.7 kB)
  Using cached tabulate-0.8.10-py3-none-any.whl.metadata (25 kB)
  Using cached pytz-2025.1-py2.py3-none-any.whl.metadata (22 kB)
  Using cached tzlocal-5.2-py3-none-any.whl.metadata (7.8 kB)
Using cached wbdata-1.0.0-py3-none-any.whl (18 kB)
Using cached appdirs-1.4.4-py2.py3-none-any.whl (9.6 kB)
Using cached backoff-2.2.1-py3-none-any.whl (15 kB)
Using cached dateparser-1.2.1-py3-none-any.whl (295 kB)
Using cached shelved_cache-0.3.1-py3-none-any.whl (7.8 kB)
Using cached tabulate-0.8.10-py3-none-any.whl (29 kB)
Using cached pytz-2025.1-py2.py3-none-any.whl (507 kB)
Using cached tzlocal-5.2-py3-none-any.whl (17 kB)
  Attempting uninstall: pytz
    Found exist

Key '-7695621915466402873' not in persistent cache.
Key '-5016343606866289725' not in persistent cache.
Key '2646845878116636709' not in persistent cache.
Key '8598485012572243195' not in persistent cache.
Key '-880113895970335847' not in persistent cache.
Key '8951414286618048258' not in persistent cache.
Key '7750803547085383787' not in persistent cache.
Key '-410583629554511385' not in persistent cache.
Key '-7176871012076283560' not in persistent cache.
Key '3108485884993372884' not in persistent cache.
Key '44015540799312144' not in persistent cache.
Key '-1781237132843978292' not in persistent cache.
Key '7210675039947924130' not in persistent cache.
Key '-4700926946760897743' not in persistent cache.
Key '4417110690304019125' not in persistent cache.
Key '5838667362858084664' not in persistent cache.
Key '6001998844724881934' not in persistent cache.
Key '-9179913126986111376' not in persistent cache.
Key '-8925675714436785413' not in persistent cache.
Key '-6246397405836672265'

  Using cached cufflinks-0.17.3-py3-none-any.whl
  Using cached colorlover-0.3.0-py3-none-any.whl.metadata (421 bytes)
Using cached colorlover-0.3.0-py3-none-any.whl (8.9 kB)
Note: you may need to restart the kernel to use updated packages.


ModuleNotFoundError: No module named 'rasterio'

In [5]:
wbdata.get_countries()

Key '8483189953449081214' not in persistent cache.


id    name
----  --------------------------------------------------------------------------------
ABW   Aruba
AFE   Africa Eastern and Southern
AFG   Afghanistan
AFR   Africa
AFW   Africa Western and Central
AGO   Angola
ALB   Albania
AND   Andorra
ARB   Arab World
ARE   United Arab Emirates
ARG   Argentina
ARM   Armenia
ASM   American Samoa
ATG   Antigua and Barbuda
AUS   Australia
AUT   Austria
AZE   Azerbaijan
BDI   Burundi
BEA   East Asia & Pacific (IBRD-only countries)
BEC   Europe & Central Asia (IBRD-only countries)
BEL   Belgium
BEN   Benin
BFA   Burkina Faso
BGD   Bangladesh
BGR   Bulgaria
BHI   IBRD countries classified as high income
BHR   Bahrain
BHS   Bahamas, The
BIH   Bosnia and Herzegovina
BLA   Latin America & the Caribbean (IBRD-only countries)
BLR   Belarus
BLZ   Belize
BMN   Middle East & North Africa (IBRD-only countries)
BMU   Bermuda
BOL   Bolivia
BRA   Brazil
BRB   Barbados
BRN   Brunei Darussalam
BSS   Sub-Saharan Africa (IBRD-only countries)
BTN   Bhutan
BWA  

In [10]:
wbdata.get_sources
SOURCE = 40 # "Population estimates and projections

indicators = wbdata.get_indicators(source=SOURCE)

In [11]:
source = 40 # "Population estimates and projections

indicators = wbdata.get_indicators(source=source)
indicators

id                 name
-----------------  -------------------------------------------------------------------
SH.DTH.0509        Number of deaths ages 5-9 years
SH.DTH.0514        Number of deaths ages 5-14 years
SH.DTH.1014        Number of deaths ages 10-14 years
SH.DTH.1019        Number of deaths ages 10-19 years
SH.DTH.1519        Number of deaths ages 15-19 years
SH.DTH.2024        Number of deaths ages 20-24 years
SH.DTH.IMRT        Number of infant deaths
SH.DTH.IMRT.FE     Number of infant deaths, female
SH.DTH.IMRT.MA     Number of infant deaths, male
SH.DTH.MORT        Number of under-five deaths
SH.DTH.MORT.FE     Number of under-five deaths, female
SH.DTH.MORT.MA     Number of under-five deaths, male
SH.DTH.NMRT        Number of neonatal deaths
SH.DYN.0509        Probability of dying among children ages 5-9 years (per 1,000)
SH.DYN.0514        Probability of dying at age 5-14 years (per 1,000 children age 5)
SH.DYN.1014        Probability of dying among adolescents ages 1

A function that returns a pandas DataFrame indexed by Region or Country and Year, with columns giving counts of people in different age-sex groups.


In [39]:

# Function to retrieve population data from World Bank
def get_population_data(year, variables, gender, place_code):
    population_data = wbdata.get_dataframe(variables, country=place_code)
    population_data.index = population_data.index.astype(int)
    population_data.reset_index(inplace=True)
    population_data = population_data[population_data['date'] == year].melt(
        id_vars=["date"], var_name="age", value_name=gender
    )
    # Add place code 
    population_data['place'] = place_code
    population_data.set_index(['place', 'date'], inplace=True)
    return population_data

# Create five-year age groups
def age_list(age_range):
    def round_age(age_range):
        for i in range(0, 2):
            if 0 < age_range[i] % 5 < 3:
                age_range[i] -= age_range[i] % 5
            elif age_range[i] % 5 >= 3:
                age_range[i] += (5 - age_range[i] % 5)
        return age_range

    results = []
    age_lb, age_ub = round_age(age_range)
    while age_lb < min(79, age_ub):
        results.append("{:02d}{:02d}".format(age_lb, age_lb + 4))
        age_lb += 5
    if age_lb >= 80:
        results.append('80UP')
    return results 

# Function to create a population DataFrame
def population_df(year, sex, age_range, place_code='WLD'):
    age_ranges = age_list(age_range)
    male_variables = {"SP.POP." + i + ".MA": "{}-{}".format(i[:2], i[2:]) for i in age_ranges}
    female_variables = {"SP.POP." + i + ".FE": "{}-{}".format(i[:2], i[2:]) for i in age_ranges}
    
    male_population = get_population_data(year, male_variables, 'male', place_code)
    female_population = get_population_data(year, female_variables, 'female', place_code)
    
    # Merge data based on selected sex
    if sex == 'male':
        df = male_population
    elif sex == 'female':
        df = female_population
    else:
        # Combine male and female data for total population
        df = male_population.merge(female_population, on="age").set_axis(male_population.index)
        df["total"] = df["female"] + df["male"]
    
    return df

In [47]:
population_df(year=(2020), sex='all', age_range=[0, 100], place_code='USA')

age        male      female       total
place date                                           
USA   2020  00-04   9698499.0   9249649.0  18948148.0
      2020  05-09  10407741.0   9845524.0  20253265.0
      2020  10-14  11211805.0  10564057.0  21775862.0
      2020  15-19  11171654.0  10471497.0  21643151.0
      2020  20-24  11255580.0  10485189.0  21740769.0
      2020  25-29  12044675.0  11208797.0  23253472.0
      2020  30-34  12140507.0  11329689.0  23470196.0
      2020  35-39  11684006.0  10977860.0  22661866.0
      2020  40-44  10795498.0  10235337.0  21030835.0
      2020  45-49  10345587.0   9892260.0  20237847.0
      2020  50-54  10503469.0  10173335.0  20676804.0
      2020  55-59  10972096.0  10858519.0  21830615.0
      2020  60-64  10220609.0  10500778.0  20721387.0
      2020  65-69   8452014.0   9080960.0  17532974.0
      2020  70-74   6702890.0   7473892.0  14176782.0
      2020  75-79   4341761.0   5189699.0   9531460.0
      2020  80-UP   4783983.0   7257518.0  12041501.0

In [50]:
population_df(year=(2000), sex='male', age_range=[0, 50], place_code='TUV')

age   male
place date              
TUV   2000  00-04  629.0
      2000  05-09  626.0
      2000  10-14  580.0
      2000  15-19  440.0
      2000  20-24  308.0
      2000  25-29  271.0
      2000  30-34  336.0
      2000  35-39  334.0
      2000  40-44  281.0
      2000  45-49  229.0